In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.metrics import log_loss
from sklearn.naive_bayes import BernoulliNB
from sklearn.linear_model import LogisticRegression
from tensorflow.keras.utils import to_categorical
import numpy as np
 
#Load Data with pandas, and parse the first column into datetime
# train=pd.read_csv('train.csv', parse_dates = ['Dates'])
# test=pd.read_csv('test.csv', parse_dates = ['Dates'])
data=pd.read_csv('data/2018_crime_weather_data.csv')
data.head()

,Unnamed: 0,X,Y,CCN,REPORT_DAT,SHIFT,METHOD,OFFENSE,BLOCK,XBLOCK,...,BID,START_DATE,END_DATE,OBJECTID,OCTO_RECORD_ID,DATE,AVG,MAX,MIN,PctIllum
0,0,-77.074960,38.912611,10954295,2018-04-05T21:17:59.000Z,EVENING,OTHERS,SEX ABUSE,3810 - 3899 BLOCK OF RESERVOIR ROAD NW,393499,...,NaN,2018-04-01T00:01:17.000Z,2018-04-05T21:18:34.000Z,313778329,10954295-01,2018-04-05,42.000000,54.0,32.0,0.78
1,1,-76.978422,38.899255,12146972,2018-07-31T00:00:00.000Z,MIDNIGHT,OTHERS,HOMICIDE,1700 - 1799 BLOCK OF BENNING ROAD NE,401872,...,NaN,2012-10-18T00:10:00.000Z,2012-10-18T00:12:00.000Z,313838908,12146972-01,2018-07-31,72.750000,81.0,69.0,0.90
2,2,-76.929229,38.901839,16208156,2018-01-05T00:00:00.000Z,MIDNIGHT,GUN,HOMICIDE,4916 - 5067 BLOCK OF JAY STREET NE,406139,...,NaN,2016-12-09T09:07:12.000Z,2016-12-09T09:11:51.000Z,313989396,16208156-01,2018-01-05,14.285714,19.0,10.0,0.86
3,3,-76.965614,38.851265,17003498,2018-03-20T00:00:00.000Z,MIDNIGHT,GUN,HOMICIDE,3030 - 3299 BLOCK OF BUENA VISTA TERRACE SE,402985,...,NaN,2017-01-06T19:01:52.000Z,2017-01-06T19:10:39.000Z,313996993,17003498-01,2018-03-20,34.745098,48.0,30.0,0.08
4,4,-76.984578,38.898121,17009136,2018-07-09T00:00:00.000Z,MIDNIGHT,GUN,HOMICIDE,600 - 699 BLOCK OF 14TH PLACE NE,401338,...,NaN,2017-01-17T06:38:02.000Z,NaN,313999072,17009136-02,2018-07-09,72.035714,87.0,55.0,0.21


In [2]:
data.drop(data.columns[[0]], axis=1, inplace=True)
data.head()

,X,Y,CCN,REPORT_DAT,SHIFT,METHOD,OFFENSE,BLOCK,XBLOCK,YBLOCK,...,BID,START_DATE,END_DATE,OBJECTID,OCTO_RECORD_ID,DATE,AVG,MAX,MIN,PctIllum
0,-77.074960,38.912611,10954295,2018-04-05T21:17:59.000Z,EVENING,OTHERS,SEX ABUSE,3810 - 3899 BLOCK OF RESERVOIR ROAD NW,393499,138307,...,NaN,2018-04-01T00:01:17.000Z,2018-04-05T21:18:34.000Z,313778329,10954295-01,2018-04-05,42.000000,54.0,32.0,0.78
1,-76.978422,38.899255,12146972,2018-07-31T00:00:00.000Z,MIDNIGHT,OTHERS,HOMICIDE,1700 - 1799 BLOCK OF BENNING ROAD NE,401872,136822,...,NaN,2012-10-18T00:10:00.000Z,2012-10-18T00:12:00.000Z,313838908,12146972-01,2018-07-31,72.750000,81.0,69.0,0.90
2,-76.929229,38.901839,16208156,2018-01-05T00:00:00.000Z,MIDNIGHT,GUN,HOMICIDE,4916 - 5067 BLOCK OF JAY STREET NE,406139,137111,...,NaN,2016-12-09T09:07:12.000Z,2016-12-09T09:11:51.000Z,313989396,16208156-01,2018-01-05,14.285714,19.0,10.0,0.86
3,-76.965614,38.851265,17003498,2018-03-20T00:00:00.000Z,MIDNIGHT,GUN,HOMICIDE,3030 - 3299 BLOCK OF BUENA VISTA TERRACE SE,402985,131495,...,NaN,2017-01-06T19:01:52.000Z,2017-01-06T19:10:39.000Z,313996993,17003498-01,2018-03-20,34.745098,48.0,30.0,0.08
4,-76.984578,38.898121,17009136,2018-07-09T00:00:00.000Z,MIDNIGHT,GUN,HOMICIDE,600 - 699 BLOCK OF 14TH PLACE NE,401338,136696,...,NaN,2017-01-17T06:38:02.000Z,NaN,313999072,17009136-02,2018-07-09,72.035714,87.0,55.0,0.21


In [3]:
data.REPORT_DAT.dtype

dtype('O')

In [4]:
data['REPORT_DAT']= data['REPORT_DAT'].astype(str)

In [5]:
data['REPORT_DAT']= data['REPORT_DAT'].map(lambda x: str(x)[:-14])

In [6]:
data.head()

,X,Y,CCN,REPORT_DAT,SHIFT,METHOD,OFFENSE,BLOCK,XBLOCK,YBLOCK,...,BID,START_DATE,END_DATE,OBJECTID,OCTO_RECORD_ID,DATE,AVG,MAX,MIN,PctIllum
0,-77.074960,38.912611,10954295,2018-04-05,EVENING,OTHERS,SEX ABUSE,3810 - 3899 BLOCK OF RESERVOIR ROAD NW,393499,138307,...,NaN,2018-04-01T00:01:17.000Z,2018-04-05T21:18:34.000Z,313778329,10954295-01,2018-04-05,42.000000,54.0,32.0,0.78
1,-76.978422,38.899255,12146972,2018-07-31,MIDNIGHT,OTHERS,HOMICIDE,1700 - 1799 BLOCK OF BENNING ROAD NE,401872,136822,...,NaN,2012-10-18T00:10:00.000Z,2012-10-18T00:12:00.000Z,313838908,12146972-01,2018-07-31,72.750000,81.0,69.0,0.90
2,-76.929229,38.901839,16208156,2018-01-05,MIDNIGHT,GUN,HOMICIDE,4916 - 5067 BLOCK OF JAY STREET NE,406139,137111,...,NaN,2016-12-09T09:07:12.000Z,2016-12-09T09:11:51.000Z,313989396,16208156-01,2018-01-05,14.285714,19.0,10.0,0.86
3,-76.965614,38.851265,17003498,2018-03-20,MIDNIGHT,GUN,HOMICIDE,3030 - 3299 BLOCK OF BUENA VISTA TERRACE SE,402985,131495,...,NaN,2017-01-06T19:01:52.000Z,2017-01-06T19:10:39.000Z,313996993,17003498-01,2018-03-20,34.745098,48.0,30.0,0.08
4,-76.984578,38.898121,17009136,2018-07-09,MIDNIGHT,GUN,HOMICIDE,600 - 699 BLOCK OF 14TH PLACE NE,401338,136696,...,NaN,2017-01-17T06:38:02.000Z,NaN,313999072,17009136-02,2018-07-09,72.035714,87.0,55.0,0.21


In [7]:
new = data["REPORT_DAT"].str.split("-", expand = True)
new.head()

,0,1,2
0,2018,04,05
1,2018,07,31
2,2018,01,05
3,2018,03,20
4,2018,07,09


In [8]:
data["report_year"] = new[0]
data["report_month"] = new[1]
data["report_day"] = new[2]
data.head()

,X,Y,CCN,REPORT_DAT,SHIFT,METHOD,OFFENSE,BLOCK,XBLOCK,YBLOCK,...,OBJECTID,OCTO_RECORD_ID,DATE,AVG,MAX,MIN,PctIllum,report_year,report_month,report_day
0,-77.074960,38.912611,10954295,2018-04-05,EVENING,OTHERS,SEX ABUSE,3810 - 3899 BLOCK OF RESERVOIR ROAD NW,393499,138307,...,313778329,10954295-01,2018-04-05,42.000000,54.0,32.0,0.78,2018,04,05
1,-76.978422,38.899255,12146972,2018-07-31,MIDNIGHT,OTHERS,HOMICIDE,1700 - 1799 BLOCK OF BENNING ROAD NE,401872,136822,...,313838908,12146972-01,2018-07-31,72.750000,81.0,69.0,0.90,2018,07,31
2,-76.929229,38.901839,16208156,2018-01-05,MIDNIGHT,GUN,HOMICIDE,4916 - 5067 BLOCK OF JAY STREET NE,406139,137111,...,313989396,16208156-01,2018-01-05,14.285714,19.0,10.0,0.86,2018,01,05
3,-76.965614,38.851265,17003498,2018-03-20,MIDNIGHT,GUN,HOMICIDE,3030 - 3299 BLOCK OF BUENA VISTA TERRACE SE,402985,131495,...,313996993,17003498-01,2018-03-20,34.745098,48.0,30.0,0.08,2018,03,20
4,-76.984578,38.898121,17009136,2018-07-09,MIDNIGHT,GUN,HOMICIDE,600 - 699 BLOCK OF 14TH PLACE NE,401338,136696,...,313999072,17009136-02,2018-07-09,72.035714,87.0,55.0,0.21,2018,07,09


In [13]:
crime_count_year = data.groupby('report_year')['CCN'].nunique()

In [14]:
crime_count_year

report_year
2018    33635
Name: CCN, dtype: int64

In [17]:
crime_count_month = data.groupby('report_month')['CCN'].nunique()
crime_count_month

report_month
01    2531
02    2185
03    2304
04    2407
05    2728
06    2839
07    3137
08    3326
09    3161
10    3279
11    2850
12    2889
Name: CCN, dtype: int64

In [18]:
crime_count_day = data.groupby('report_day')['CCN'].nunique()
crime_count_day

report_day
01    1004
02    1003
03    1004
04     987
05    1011
06    1014
07    1088
08    1079
09    1138
10    1068
11    1080
12    1095
13    1129
14    1155
15    1110
16    1218
17    1165
18    1063
19    1156
20    1219
21    1079
22    1122
23    1193
24    1074
25    1087
26    1168
27    1170
28    1210
29    1084
30    1010
31     654
Name: CCN, dtype: int64